In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedGroupKFold
from sklearn.metrics import make_scorer, balanced_accuracy_score
import joblib
import gzip
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, confusion_matrix, accuracy_score
import json
import os
import numpy as np
import pickle

In [4]:
test_data =pd.read_csv("../files/input/test_data.csv.zip") #Leemos el dataframe de test_data.csv.zip
train_data =pd.read_csv("../files/input/train_data.csv.zip") #Leemos el dataframe de test_data.csv.zip


In [5]:
# Paso 1. Realizar  limpieza de los datasets:
test_data.rename(columns={"default payment next month": "default"}, inplace=True) # Renombramos la columna para test_data
test_data.drop(columns=["ID"], inplace=True) # Eliminar la columna 'ID'
test_data.dropna(inplace=True)  # Eliminar filas con cualquier valor nulo
test_data["EDUCATION"] = test_data["EDUCATION"].apply(lambda x: 4 if x > 4 else x)
test_data.query('MARRIAGE > 0 and EDUCATION > 0')

train_data.rename(columns={"default payment next month": "default"}, inplace=True) # Renombramos la columna train_data
train_data.drop(columns=["ID"], inplace=True) # Eliminar la columna 'ID'
train_data.dropna(inplace=True) # Eliminar filas con cualquier valor nulo
train_data["EDUCATION"] = train_data["EDUCATION"].apply(lambda x: 4 if x > 4 else x)
train_data.query('MARRIAGE > 0 and EDUCATION > 0')


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,310000,1,3,1,32,0,0,0,0,0,0,172772,152397,110375,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
1,10000,2,3,1,49,-1,-1,-2,-1,2,2,32,-358,-748,1690,1138,930,0,0,2828,0,182,0,1
2,50000,1,2,1,28,-1,-1,-1,0,-1,-1,430,0,46257,45975,1300,43987,0,46257,2200,1300,43987,1386,0
3,80000,2,3,1,52,2,2,3,3,3,2,36649,39448,40101,40748,39816,40607,3700,1600,1600,0,1600,1600,1
4,270000,1,1,2,34,1,2,0,0,2,0,20979,17228,20924,22448,15490,17343,0,4000,2000,0,2000,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,140000,2,2,1,27,2,-1,-1,-1,0,0,776,752,776,1580,804,728,752,800,1580,0,700,700,0
20996,130000,1,2,2,41,0,0,0,0,0,0,124391,126032,123746,123107,42897,39378,4442,5200,5012,2500,5000,2000,0
20997,50000,1,3,2,23,0,0,0,0,0,0,47620,41810,36023,28967,29829,30046,1973,1426,1001,1432,1062,997,0
20998,90000,2,3,2,25,0,0,0,0,0,0,16977,19612,5613,5613,10113,10113,3000,3000,0,4500,0,3440,0


In [6]:
# Paso 2. Dividr los datasets en x_train, y_train, x_test, y_test.
# Definir las características (X) y la variable objetivo (y)
X_train = train_data.drop(columns=["default"])  # Todas las columnas excepto "default"
y_train = train_data["default"]  # Columna objetivo

X_test = test_data.drop(columns=["default"])  # Todas las columnas excepto "default"
y_test = test_data["default"]  # Columna objetivo


In [8]:
#Paso 3.# Crear un pipeline para el modelo de clasificación
# Identificar columnas categóricas
categorical_features = ["SEX", "EDUCATION", "MARRIAGE"]
# categorical_transformer = OneHotEncoder(handle_unknown="ignore")

# Crear un preprocesador para transformar las variables categóricas
transformer = ColumnTransformer(
    transformers=[
        ("ohe", OneHotEncoder(dtype=int), categorical_features)
    ],
    remainder='passthrough'  
)

pipeline = Pipeline(steps=[
    ('transformer', transformer),
    ('clasi', RandomForestClassifier(n_jobs=-1, random_state=17))
])


# preprocessor = ColumnTransformer(
#     transformers=[
#         ("cat", categorical_transformer, categorical_features)
#     ],
#     remainder="passthrough"  
# )

# rf = RandomForestClassifier(random_state=42)

# # Crear el pipeline
# pipeline = Pipeline(steps=[
#     ("preprocessor", preprocessor),  # Transformar variables categóricas
#     ("classifier", rf)  # Modelo Random Forest
# ])

pipeline
# Ajustar el pipeline al conjunto de entrenamiento
pipeline.fit(X_train, y_train)
print("Precisión:", pipeline.score(X_test, y_test))

# # Evaluar el modelo en el conjunto de prueba
# accuracy = pipeline.score(X_test, y_test)
# print(f"Accuracy del modelo: {accuracy:.2f}")

Precisión: 0.8264444444444444


In [19]:
# Paso 4.Optimice los hiperparametros del pipeline usando validación cruzada.

# Definir los hiperparámetros correctamente
param_grid = {
    'clasi__n_estimators': [180],
    'clasi__max_features': ['sqrt'],
    'clasi__min_samples_split': [10],
    'clasi__min_samples_leaf': [2],
    'clasi__bootstrap': [True],
    'clasi__max_depth': [None]
}

# Crear el objeto de GridSearchCV
grid_search = GridSearchCV(
    pipeline,  # Pipeline definido previamente
    param_grid=param_grid,  # Hiperparámetros a optimizar
    cv=10,  # Número de splits para la validación cruzada
    scoring= 'balanced_accuracy' ,  # Métrica de precisión balanceada
    verbose=1,  # Mostrar progreso
    n_jobs=-1,  # Usar todos los núcleos disponibles
    refit=True
)

# Ajustar GridSearchCV con los datos de entrenamiento
grid_search.fit(X_train, y_train)



Fitting 10 folds for each of 1 candidates, totalling 10 fits


c:\Users\aguir\Documentos\ANALITICA PREDICTIVA\2024-2-LAB-01-prediccion-del-default-usando-rf-Luatorresag\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:1651: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(dtype=<class 'int'>),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE'])])),
                                       ('clasi',
                                        RandomForestClassifier(n_jobs=-1,
                                                               random_state=17))]),
             n_jobs=-1,
             param_grid={'clasi__bootstrap': [True], 'clasi__max_depth': [None],
                         'clasi__max_features': ['sqrt'],
                         'clasi__min_samples_leaf': [2],
                         'clasi__min_samples_split': [10],
                         'clasi__n_estimators': [180]},
             scoring='balanced_accuracy', verbose=1)

In [34]:
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
os.makedirs('../files/models', exist_ok=True)


with gzip.open('../files/models/model.pkl.gz', 'wb') as file:
    pickle.dump(grid_search, file)

def cargar_modelo_y_predecir(data, modelo_path="../files/models/model.pkl.gz"):
    try:
        with gzip.open(modelo_path, "rb") as file:
            estimator = pickle.load(file)
        return estimator.predict(data)
    except FileNotFoundError:
        raise FileNotFoundError(f"No se encontró el archivo de modelo en la ruta especificada: {modelo_path}")
    except Exception as e:
        raise RuntimeError(f"Error al cargar el modelo o realizar predicciones: {e}")

# Uso de la función
y_train_pred = cargar_modelo_y_predecir(X_train)
y_test_pred = cargar_modelo_y_predecir(X_test)

In [36]:
def metricas(dict_metricas):
    models_dir = '../files/output'
    os.makedirs(models_dir, exist_ok=True)

    if os.path.exists('../files/output/metrics.json'):
        with open('../files/output/metrics.json', mode='r') as file:
            if len(file.readlines()) >= 4:
                os.remove('../files/output/metrics.json')

    with open('../files/output/metrics.json', mode='a') as file:
        file.write(str(dict_metricas).replace("'", '"') + "\n")


def evaluacion(dataset, y_true, y_pred):
    accuracy = float(accuracy_score(y_true, y_pred))
    precision = float(precision_score(y_true, y_pred))
    balanced_accuracy = float(balanced_accuracy_score(y_true, y_pred))
    recall = float(recall_score(y_true, y_pred))
    f1 = float(f1_score(y_true, y_pred))
    metrics = {
        "type": "metrics",
        "dataset": dataset,
        "precision": precision,
        "balanced_accuracy": balanced_accuracy,
        "recall": recall,
        "f1_score": f1
    }

    metricas(metrics)

metrics_train = evaluacion('train', y_train, y_train_pred)
metrics_test = evaluacion('test', y_test, y_test_pred)

def matriz_confusion(dataset, y_true, y_pred):
    matriz = confusion_matrix(y_true, y_pred)
    matrix_confusion = {
        "type": "cm_matrix",
        "dataset": dataset,
        "true_0": {
            "predicted_0": int(matriz[0, 0]),
            "predicted_1": int(matriz[0, 1]),
        },
        "true_1": {
            "predicted_0": int(matriz[1, 0]),
            "predicted_1": int(matriz[1, 1])
        }
    }

    metricas(json.dumps(matrix_confusion))


metrics_train_cm = matriz_confusion('train', y_train, y_train_pred)
metrics_test_cm = matriz_confusion('test', y_test, y_test_pred)

In [29]:
# # Paso 6. Calcule las metricas de precision, precision balanceada, recall y f1-score para los conjuntos de entrenamiento y prueba.
   
# # Predicciones en los conjuntos de entrenamiento y prueba
# y_train_pred = grid_search.predict(X_train)
# y_test_pred = grid_search.predict(X_test)


# # Métricas para el conjunto de entrenamiento
# metrics = [
#     {
#         'dataset': 'train',
#         'precision': precision_score(y_train, y_train_pred),
#         'recall' :  recall_score(y_train, y_train_pred),
#         'f1_score' : f1_score(y_train, y_train_pred),
#         'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred)
#     },
#     {
#         'dataset': 'test',
#         'precision': precision_score(y_test, y_test_pred),
#         'recall' :  recall_score(y_test, y_test_pred),
#         'f1_score' : f1_score(y_test, y_test_pred),
#         'balanced_accuracy': balanced_accuracy_score(y_test, y_test_pred)
#     }
# ]

# # with open('../files/output/metrics.json', 'w') as f:
# #     json.dump(metrics, f, indent=4)

In [28]:
# confusion_matrices = [
#     {
#         'type': 'cm_matrix', 
#         'dataset': 'train', 
#         'true_0':{"predicted_0": int(train_cm[0,0]), 
#                    "predicte_1": int(train_cm[0,1])}, 
#         'true_1':{"predicted_0": int(train_cm[1,0]), 
#                     "predicted_1": int(train_cm[1,1])}
#     },
#     {
#         'type': 'cm_matrix', 
#         'dataset': 'test', 
#         'true_0':{"predicted_0": int(test_cm[0,0]), 
#                    "predicte_1": int(test_cm[0,1])}, 
#         'true_1':{"predicted_0": int(test_cm[1,0]), 
#                     "predicted_1": int(test_cm[1,1])}
#     }
# ]

# with open('../files/output/metrics.json', 'r') as f:
#     data = json.load(f)

# data = data + confusion_matrices 

# with open('../files/output/metrics.json', 'w') as f:
#     json.dump(data, f, indent=4)
